In [2]:
import pandas as pd

df = pd.read_excel("advisory_data_file_excel.xlsx", sheet_name="IC Master")

In [3]:
df.columns = df.columns.str.strip()
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [4]:
crop_names = list(df['crop'].unique())
print(crop_names)

['Redgram', 'Clusterbean', 'Paddy', 'Soyabean', 'Sugarcane Seasonal', 'Maize', 'Tomato', 'Groundnut', 'Cotton', 'Bajra', 'Banana', 'Chilli', 'Sugarcane Adsali', 'Cauliflower', 'Lentil', 'Onion', 'Coriander', 'Brinjal', 'Garlic', 'Cabbage', 'Mustard', 'Okra', 'Chickpea', 'Potato', 'Pea', 'Wheat']


In [5]:
tasks_by_crop = (
    df.groupby('crop')
    .apply(lambda x: x.loc[x['state'] == x['state'].iloc[0], 'title'])
    .reset_index(name='title')
)

In [6]:
tasks_by_crop["description"] = ""

In [7]:
tasks_by_crop

,crop,level_1,title,description
0,Bajra,14,Soil Sample Collection,
1,Bajra,22,Soil Test Report,
2,Bajra,37,Variety Selection,
3,Bajra,58,Primary Ploughing,
4,Bajra,77,Organic Manure Application,
...,...,...,...,...
5656,Wheat,18836,Irrigation 4,
5657,Wheat,18837,Spray 4 Disease Nutrition,
5658,Wheat,18838,Irrigation 5,
5659,Wheat,18839,Irrigation 6,


In [8]:
df_2 = pd.read_excel("advisory_data_file_excel.xlsx", sheet_name="Title Description")

In [9]:
df_2.columns = df_2.columns.str.strip()

In [10]:
df_2 = df_2.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [11]:
for i in range(len(tasks_by_crop)):
    row = tasks_by_crop.iloc[i]
    crop = row.crop
    task_title = row.title
    try:
        description = df_2[(df_2["crop"]==crop) & (df_2["title"]==task_title)].iloc[0]["description"]
    except IndexError:
        print("Index Error for")
        print(crop, task_title)
        description = ""
    tasks_by_crop.at[i, "description"] = description
print(tasks_by_crop)

       crop  level_1                       title  \
0     Bajra       14      Soil Sample Collection   
1     Bajra       22            Soil Test Report   
2     Bajra       37           Variety Selection   
3     Bajra       58           Primary Ploughing   
4     Bajra       77  Organic Manure Application   
...     ...      ...                         ...   
5656  Wheat    18836                Irrigation 4   
5657  Wheat    18837   Spray 4 Disease Nutrition   
5658  Wheat    18838                Irrigation 5   
5659  Wheat    18839                Irrigation 6   
5660  Wheat    18840                  Harvesting   

                                            description  
0     Collect the soil samples after harvest of prev...  
1     Generate the soil test report and make it avai...  
2     Select any promising high yielding varieties a...  
3     Plough the field with Power Harrow or Disk Har...  
4     Apply 5 tones well-decomposed organic manure  ...  
...                        

In [12]:
tasks_by_crop.drop(labels=["level_1"], axis=1, inplace=True)

In [14]:
tasks_by_crop.to_csv("source.csv", index=False)

In [29]:
tmp_df:pd.DataFrame = tasks_by_crop
len(tmp_df)

5661

In [30]:
tmp_df.drop_duplicates(subset=["crop", "title"], inplace=True)
len(tmp_df)

863

In [32]:
tmp_df.to_csv("source_2.csv", index=False)

In [40]:
data = tmp_df.groupby('crop').apply(lambda x: x[['title', 'description']].to_dict(orient='records')).to_dict()

In [58]:
final_output = ""
for key, value in data.items():
    output = f"How to grow {key}?\n\n"
    for i, task_item in enumerate(value):
        title = task_item.get("title")
        desc = task_item.get("description")
        output += f"Step {i+1} {title}\n{desc}\n\n"
    final_output += output

with open("source.txt", "w") as out:
    out.writelines(final_output)